# Clasifikasi Menggunakan Dataset CIFAR-10

[Dataset](https://github.com/YoongiKim/CIFAR-10-images)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
from torchmetrics.classification import Accuracy
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

c:\Users\dsapu\Project\conda\condasystem\envs\dmsdl\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cpu')

# dataset dan dataloader

In [3]:
bs = 256
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(32, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    ])
test_transform = transforms.Compose([
    transforms.ToTensor(),  
    ])

train_set = datasets.ImageFolder(root='CIFAR-10/train', transform=train_transform)
train_loader = DataLoader(train_set, batch_size=bs, shuffle=True, num_workers=2)
test_set = datasets.ImageFolder(root='CIFAR-10/test', transform=test_transform)
test_loader = DataLoader(test_set, batch_size=bs, shuffle=False, num_workers=2)

In [4]:
label2cat, idxclass = train_set.classes, train_set.class_to_idx
label2cat

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [5]:
feature, target = next(iter(train_loader))
feature.shape, target.shape

(torch.Size([256, 3, 32, 32]), torch.Size([256]))

# arsitektur dan config

In [6]:
def conv_block(in_feature, out_feature, padding=1, stride=1,
                  activation="relu", pool =True, maxpool=True, kernel_size=3,
                  kernel_size_pool=2, pool_stride=2)-> list[nn.Sequential]:
    layers = [nn.Conv2d(in_feature, out_feature, kernel_size=kernel_size, padding=padding, stride=stride)]
    if activation == "relu":
        layers.append(nn.ReLU())
    elif activation == "leakyrelu":
        layers.append(nn.LeakyReLU())
    elif activation == "sigmoid":
        layers.append(nn.Sigmoid())
    elif activation == "tanh":
        layers.append(nn.Tanh())
    if pool:
        if maxpool:
            layers.append(nn.MaxPool2d(kernel_size=kernel_size_pool, stride=pool_stride))
        else:
            layers.append(nn.AvgPool2d(kernel_size=kernel_size_pool, stride=pool_stride))
    else:
        layers.append(nn.Identity())
    return nn.Sequential(*layers)



def linear_block(in_features, out_features, activation=None, dropout=0.0):
    layers = [nn.Linear(in_features, out_features)]
    # if batch_norm:
    #     layers.append(BatchNorm1d(out_features))
    if activation == 'relu':
        layers.append(nn.ReLU())
    elif activation == 'sigmoid':
        layers.append(nn.Sigmoid())
    elif activation == 'tanh':
        layers.append(nn.Tanh())
    elif activation == 'leakyrelu':
        layers.append(nn.LeakyReLU())
    elif activation == 'softmax':
        layers.append(nn.Softmax(dim=1))
    elif activation == 'elu':
        layers.append(nn.ELU())
    elif activation == 'selu':
        layers.append(nn.SELU())
    elif activation == 'lsoftmax':
        layers.append(nn.LogSoftmax(dim=1))
    if dropout > 0.0:
        layers.append(nn.Dropout(dropout))
    return nn.Sequential(*layers)


In [7]:
class CNN(pl.LightningModule):
    def __init__(self, num_classes, dropout=0.0):
        super(CNN, self).__init__()
        self.save_hyperparameters()
        self.conv = nn.Sequential(
            conv_block(3, 8),
            conv_block(8, 16),
            conv_block(16, 32),
            nn.Flatten()
        )
        self.fc = nn.Sequential(
            linear_block(32 * 4 * 4, 128, dropout=dropout, activation='relu'),
            linear_block(128, 64, dropout=dropout, activation='relu'),
            linear_block(64, len(num_classes)),
        )
        self.criterion = nn.CrossEntropyLoss()
        self.accuracy = Accuracy(task='multiclass', num_classes=len(num_classes))
    def forward(self, x):
        return self.fc(self.conv(x))
    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        loss = self.criterion(outputs, labels)
        acc = self.accuracy(outputs, labels)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss
    def validation_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        loss = self.criterion(outputs, labels)
        acc = self.accuracy(outputs, labels)
        self.log('val_loss', loss, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_acc', acc, on_epoch=True, prog_bar=True, logger=True)
        return {'val_loss': loss, 'val_acc': acc}
    def configure_optimizers(self):
        optimizer = optim.RAdam(self.parameters(), lr=0.005)
        return optimizer

In [8]:
model = CNN(label2cat, dropout=0.1).to(device)
criterion = nn.NLLLoss()
optimizer = optim.RAdam(model.parameters(), lr=0.005)

In [9]:
best_val_accuracy = 0.0 
patience = 5
epochs_no_improve = 0

In [10]:
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='checkpoints/',
    filename='cifar10-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,
    mode='min'
)
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    mode='min',
)
lr_monitor_callback = LearningRateMonitor(logging_interval='epoch')

trainer = pl.Trainer(
    max_epochs=50,
    callbacks=[checkpoint_callback, early_stopping, lr_monitor_callback],
    logger=pl.loggers.TensorBoardLogger("tb_logs", name="simple_model_experiment"), # Untuk visualisasi
    accelerator='auto',
    log_every_n_steps=10)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [11]:
print("\nMemulai pelatihan CNN dengan PyTorch Lightning...")
trainer.fit(model, train_loader, test_loader)

print("\nPelatihan CNN selesai dengan PyTorch Lightning.")
print(f"Model CNN terbaik disimpan di: {checkpoint_callback.best_model_path}")

c:\Users\dsapu\Project\conda\condasystem\envs\dmsdl\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:658: Checkpoint directory C:\Users\dsapu\Project\Python\repo\CIFAR-10\Code\checkpoints exists and is not empty.

  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | conv      | Sequential         | 6.0 K  | train
1 | fc        | Sequential         | 74.6 K | train
2 | criterion | CrossEntropyLoss   | 0      | train
3 | accuracy  | MulticlassAccuracy | 0      | train
---------------------------------------------------------
80.6 K    Trainable params
0         Non-trainable params
80.6 K    Total params
0.322     Total estimated model params size (MB)
27        Modules in train mode
0         Modules in eval mode



Memulai pelatihan CNN dengan PyTorch Lightning...
Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\dsapu\Project\conda\condasystem\envs\dmsdl\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


c:\Users\dsapu\Project\conda\condasystem\envs\dmsdl\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Epoch 3:  40%|███▉      | 78/196 [00:17<00:26,  4.47it/s, v_num=4, train_loss_step=1.560, train_acc_step=0.430, val_loss=1.480, val_acc=0.468, train_loss_epoch=1.630, train_acc_epoch=0.410] 


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined